In [ ]:
%pip install openai sqlalchemy pyodbc pandas
%pip install sqlalchemy
%pip install matplotlib seaborn
%pip install flask pyodbc pandas sqlalchemy openai ngrok pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.1 MB/s eta 0:00:00


## Test DB Coneection


In [ ]:
# Install the ODBC Driver 17 for SQL Server
!curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
!curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
!apt-get update
!ACCEPT_EULA=Y apt-get install -y msodbcsql17
!apt-get install -y unixodbc-dev


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0   4176      0 --:--:-- --:--:-- --:--:--  4182
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    887      0 --:--:-- --:--:-- --:--:--   880
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,306 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://packages.m

In [ ]:
# Check if the ODBC Driver 17 is installed
!odbcinst -q -d -n "ODBC Driver 17 for SQL Server"


[ODBC Driver 17 for SQL Server]
Description=Microsoft ODBC Driver 17 for SQL Server
Driver=/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.10.so.6.1
UsageCount=1


In [ ]:
import pyodbc
# Connection string
server = ""
database = "SQL-LLm"
username = ""
password = ""  # Replace with your actual password
driver = "{ODBC Driver 17 for SQL Server}"
conn_str = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

try:
    # Attempt to connect to the database
    conn = pyodbc.connect(conn_str)
    print("Connection successful!")

    cursor = conn.cursor()
    cursor.execute("SELECT 1")
    row = cursor.fetchone()
    if row:
        print("Query executed successfully:", row[0])
    # Close the connection
    conn.close()

except pyodbc.Error as e:
    print("Connection failed. Error:", e)


Connection successful!
Query executed successfully: 1


In [ ]:
try:
    # Attempt to connect to the database
    conn = pyodbc.connect(conn_str)
    print("Connection successful!")

    # Execute a query to select all rows from the Sales_fact table
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Sales_fact")

    # Fetch all rows
    rows = cursor.fetchall()

    # Print the results
    for row in rows:
        print(row)

    # Close the connection
    conn.close()

except pyodbc.Error as e:
    print("Query execution failed. Error:", e)


## 2. With modification SQL


In [ ]:
from flask import Flask, request, jsonify
import os
from openai import AzureOpenAI
import pandas as pd
from sqlalchemy import create_engine
from pyngrok import ngrok

app = Flask(__name__)

# ngrok authtoken
ngrok.set_auth_token("<Ngrok Token>")

# Azure OpenAI
client = AzureOpenAI(
    api_key="<Your Azure API Key>",
    api_version="2024-02-01",
    azure_endpoint=""
)

# Azure SQL Server settings
server = ""
database = "SQL-LLm"
username = ""
password = ""
driver = "ODBC Driver 17 for SQL Server"

deployment_name = "gpt-35-turbo-instruct"

# generate SQL using Azure OpenAI
def generate_sql(question):
    prompt = f"""
You are an expert in SQL using Microsoft SQL Server assistant. Based on the given schema and relationships, generate an optimized SQL query.

Tables:
- Sales_Fact: sales_id, product_id, customer_id, time_id, quantity_sold, total_amount
- Products_Dim: product_id, product_name, category, price
- Customers_Dim: customer_id, customer_name, region, country
- Time_Dim: time_id, date, month, quarter, year

Relationships:
- Sales_Fact.product_id -> Products_Dim.product_id
- Sales_Fact.customer_id -> Customers_Dim.customer_id
- Sales_Fact.time_id -> Time_Dim.time_id

Question: {question}
SQL Query:
"""
    response = client.completions.create(
        model=deployment_name,
        prompt=prompt,
        temperature=0.1,  # Lower temperature for more deterministic results
        max_tokens=200,  # Increase max_tokens for longer queries
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None
    )
    return response.choices[0].text.strip()

# to execute SQL and return results
def execute_sql(sql_query):
    connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(' ', '+')}"
    engine = create_engine(connection_string)

    df = pd.read_sql(sql_query, engine)
    return df

# API endpoint to handle questions and modified SQL queries
@app.route('/query', methods=['POST'])
def handle_query():
    data = request.json
    question = data.get('question')
    sql_query = data.get('sql_query')  # Get the modified SQL query from the request

    if not question and not sql_query:
        return jsonify({"error": "Question or SQL query is required"}), 400

    try:
        if question:
            # Generate SQL query from the question
            sql_query = generate_sql(question)
            print("Generated SQL:", sql_query)
        elif sql_query:
            print("Executing Modified SQL:", sql_query)

        # Execute SQL query
        result = execute_sql(sql_query)

        result_json = result.to_json(orient='records')
        return jsonify({"sql_query": sql_query, "result": result_json})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    ngrok_tunnel = ngrok.connect(5000)
    print("Public URL:", ngrok_tunnel.public_url)
    app.run()


Public URL: https://1a67-35-234-51-197.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Generated SQL: SELECT TOP 5 c.customer_name, SUM(sf.total_amount) AS total_sales
FROM Sales_Fact sf
INNER JOIN Customers_Dim c ON sf.customer_id = c.customer_id
GROUP BY c.customer_name
ORDER BY total_sales DESC;


INFO:werkzeug:127.0.0.1 - - [31/Jan/2025 14:48:05] "POST /query HTTP/1.1" 200 -


Executing Modified SQL: SELECT TOP 5 c.customer_name, SUM(sf.total_amount) AS total_sales
FROM Sales_Fact sf
INNER JOIN Customers_Dim c ON sf.customer_id = c.customer_id
GROUP BY c.customer_name
ORDER BY total_sales DESC;


...

INFO:werkzeug:127.0.0.1 - - [31/Jan/2025 15:09:37] "POST /query HTTP/1.1" 200 -


Executing Modified SQL: SELECT TOP 5 SUM(sf.total_amount) AS total_sales, c.region
FROM Sales_Fact sf
INNER JOIN Customers_Dim c ON sf.customer_id = c.customer_id
INNER JOIN Time_Dim t ON sf.time_id = t.time_id
WHERE t.year >= YEAR(GETDATE()) - 3
GROUP BY c.region
ORDER BY total_sales DESC;


INFO:werkzeug:127.0.0.1 - - [31/Jan/2025 15:10:25] "POST /query HTTP/1.1" 200 -


Executing Modified SQL: SELECT TOP 5 SUM(sf.total_amount) AS total_sales, c.region
FROM Sales_Fact sf
INNER JOIN Customers_Dim c ON sf.customer_id = c.customer_id
INNER JOIN Time_Dim t ON sf.time_id = t.time_id
WHERE t.year >= YEAR(GETDATE()) - 3
GROUP BY c.region
ORDER BY total_sales DESC;


INFO:werkzeug:127.0.0.1 - - [31/Jan/2025 15:36:52] "POST /query HTTP/1.1" 500 -
